# Service Broker Demo
Následující příklad slouží k ukázce asynchronního volání na SQL Server. Alternativou může být využití služeb jako např. MSMQ.
V příkladu bude:<br/>
1. Připravena jednoduchá databáze včetně nutné konfigurace (BROKER_ENABLED)
2. Vytvořena sada objektů, které jsou nutné pro vytvoření SB instance
3. Otestováno asynchronní volání

## Příprava databáze
Vytvoření prázdné databáze pro Service Broker a její konfigurace

In [35]:
USE master
GO

DROP DATABASE IF EXISTS SBDemo
GO

CREATE DATABASE SBDemo
GO

ALTER DATABASE SBDemo SET SINGLE_USER WITH ROLLBACK IMMEDIATE
ALTER DATABASE SBDemo SET ENABLE_BROKER
ALTER DATABASE SBDemo SET MULTI_USER
GO

Commands completed successfully.

Total execution time: 00:00:00.0235781

Commands completed successfully.

Total execution time: 00:00:00.0530236

Commands completed successfully.

Total execution time: 00:00:01.2009774

Commands completed successfully.

Total execution time: 00:00:00.0776484

Databáze má *povolen* Service Broker, to znamená, že na pozadí vznikly systémové objekty (metadata) pro jeho *popis*.
V následujícím kroku začnou být tvořeny jednotlivé SB objekty. Prvním z nich je message type.

## Vytvoření Message Type
Message Type definuje formát zpráv zasílaných mezi účastníky konverzace. Message Type může být typu EMPTY, WELL_FORMED_XML nebo VALID_XML. Pro VALID_XML je třeba nejprve vytvořit XML SCHEMA COLLECTION (xsd popis formátu xml), což je databázový objekt, ve kterém je uložen obsah xsd souboru jako řetězec. My se spokojíme s WELL_FORMED_XML.
*Poznámka: použití xsd schémat urychluje zpracování velkého množstí (i malých) XML dokumentů*

In [36]:
USE SBDemo
GO

CREATE MESSAGE TYPE AsyncRequest VALIDATION = WELL_FORMED_XML
GO

Commands completed successfully.

Total execution time: 00:00:00.0194158

Commands completed successfully.

Total execution time: 00:00:00.0293518

Náš příklad bude používat jediný message type prostě pojmenovaný AsyncRequest. Když jsou definovány všechny Message Types, definuje se Contract, objekt, který říká, který z účastníků konverzace může poslat zprávu daného typu.

## Vytvoření kontraktu

In [37]:
USE SBDemo
GO

CREATE CONTRACT AsyncContract
(
AsyncRequest SENT BY INITIATOR
)
GO

Commands completed successfully.

Total execution time: 00:00:00.0205517

Commands completed successfully.

Total execution time: 00:00:00.0245930

Výše vytvořený kontrakt říká, že jediná zpráva v konverzaci bude typu AsyncRequest (definovaný Message Type), a že ji posílá INITIATOR - žadatel. Pokud by zpráv bylo v konverzaci více, tak je může posílat také TARGET (příjemce, zpracovatel požadavku), nebo ANY (kdokoliv).

## Vytvoření front a služeb
Každá zpráva (i odpověď) je zaslána do fronty. Fronta je přiřazena službě (service), která má definováno, jaké kontrakty přijímá. Takto lze vytvořit v jedné databázi více služeb pro různé kontrakty.

In [38]:
USE SBDemo
GO

-- the queue and the service to which requests come (in a communication between two instances it is created on the initiator side)
CREATE QUEUE InitiatorQueue

CREATE SERVICE InitiatorService 
ON QUEUE InitiatorQueue (AsyncContract)

-- the queue and the service from which requests depart (created on the target side, endpoints links this queue with the initiator's queue)
CREATE QUEUE TargetRequestQueue

CREATE SERVICE TargetRequestService 
ON QUEUE TargetRequestQueue (AsyncContract)
GO

Commands completed successfully.

Total execution time: 00:00:00.0208882

Commands completed successfully.

Total execution time: 00:00:00.0642181

Fronty a služby se dělají dvě - pro zaslání požadavku ze zdrojového volání, a pro přijetí požadavků cílem. Tyto fronty mohou být na různých instancích SQL Serverů, pak jsou propojeny endpointy.

## Service program
Uložená procedura, která zpracuje příchozí požadavek.

In [39]:
USE SBDemo
GO

DROP TABLE IF EXISTS ProcessedRequests
CREATE TABLE ProcessedRequests
(
    Id int not null identity primary key
    , RequestXml xml
    , TimeOfMessage datetime2 not null default(sysdatetime())
)
GO

CREATE OR ALTER PROC dbo.procServiceProgram
AS
DECLARE 
    @conversation_handle uniqueidentifier
    , @msgBody xml
    , @msgType sysname;
 
receive top (1)
@conversation_handle = conversation_handle,
@msgBody = CAST(message_body AS XML),
@msgType= message_type_name
from TargetRequestQueue
 
if @msgType = N'AsyncRequest'
begin
    insert dbo.ProcessedRequests (RequestXml) values (@msgBody)
    end conversation @conversation_handle;
end
go

Commands completed successfully.

Total execution time: 00:00:00.0216398

Commands completed successfully.

Total execution time: 00:00:00.0326444

Commands completed successfully.

Total execution time: 00:00:00.0362768

Service program vytvořený v tomto kroku se přiřadí službě. V následujícím příkladu tak, že bude spuštěn hned po příchodu zprávy.

## Automatická aktivace service programu

In [40]:
USE SBDemo
GO
alter queue TargetRequestQueue
    with activation
    ( 
      status = on,
      procedure_name = dbo.procServiceProgram,
      max_queue_readers = 1,
      execute as self
    )
go

Commands completed successfully.

Total execution time: 00:00:00.0207952

Commands completed successfully.

Total execution time: 00:00:00.0270438

## Test řešení

In [43]:
USE SBDemo
GO
SET NOCOUNT ON
declare @conversation_handle uniqueidentifier;

begin dialog conversation @conversation_handle
from service InitiatorService
to service 'TargetRequestService'
on contract AsyncContract
with encryption = off;

select @conversation_handle;

send on conversation @conversation_handle
message type AsyncRequest('<Test>Hello world!</Test>')
select 'Hello from requesting script!'
go

-- select * from [dbo].[InitiatorQueue]
-- select * from [dbo].[TargetRequestQueue]
select * from dbo.ProcessedRequests
GO

Commands completed successfully.

Total execution time: 00:00:00.0235005

Commands completed successfully.

Total execution time: 00:00:00.0423772

Commands completed successfully.

Total execution time: 00:00:00.0290802

(No column name)
4e93ada4-a256-e911-a952-000d3a45c463


Id,RequestXml,TimeOfMessage
1,<Test>Hello world!</Test>,2019-04-04 06:22:45.0616128
2,<Test>Hello world!</Test>,2019-04-04 06:26:30.3789826
3,<Test>Hello world!</Test>,2019-04-04 06:26:56.9196882


(No column name)
Hello from requesting script!
